In [1]:
import cv2
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score

In [3]:
img = cv2.imread(data['prev'].values[0], 0)

240

In [10]:
top_left = img[:img.shape[0]//2, :img.shape[1]//2]

(240, 320)

In [40]:
def fb_flow_from_df(data):
    vecs = []
    
    for row in data.itertuples():
        prev = cv2.imread(row[1], 0)
        curr = cv2.imread(row[2], 0)
        x_lim = prev.shape[0]//2
        y_lim = prev.shape[1]//2

        flow = cv2.calcOpticalFlowFarneback(prev, curr, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        dx, dy = flow[...,0], flow[...,1]
        
        dx_tl = np.mean(dx[:x_lim, :y_lim])
        dy_tl = np.mean(dy[:x_lim, :y_lim])
        
        dx_tr = np.mean(dx[x_lim:, :y_lim])
        dy_tr = np.mean(dy[x_lim:, :y_lim])
        
        dx_bl = np.mean(dx[:x_lim, y_lim:])
        dy_bl = np.mean(dy[:x_lim, y_lim:])
        
        dx_br = np.mean(dx[x_lim:, y_lim:])
        dy_br = np.mean(dy[x_lim:, y_lim:])
        
        #vecs.append([dx_tl,dy_tl,dx_tr,dy_tr,dx_bl,dy_bl,dx_br,dy_br])
        
        vecs.append([np.mean(dx), np.mean(dy)])
    return np.array(vecs)

In [2]:
data = pd.read_csv('data/paired.csv')

In [19]:
data_train = data.iloc[:1919]
data_test = data.iloc[1919:]

In [38]:
rf_model = RandomForestRegressor(n_estimators=50, n_jobs=-1, max_features='sqrt')

In [41]:
tot_X_train = fb_flow_from_df(data_train)

In [20]:
X_train = fb_flow_from_df(data_train)
y_train = data_train['speed'].values

In [39]:
cross_val_score(rf_model, X_train, y_train, scoring='neg_mean_squared_error')

/Users/adrian/anaconda3/envs/pydata/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([-18.5939125 , -10.25938612,  -6.90316527])

In [36]:
gb_model = GradientBoostingRegressor()

In [37]:
cross_val_score(gb_model, X_train, y_train, scoring='neg_mean_squared_error')

/Users/adrian/anaconda3/envs/pydata/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


array([-19.05016224, -11.89196203,  -6.97915156])

In [43]:
print(cross_val_score(rf_model, X_train, y_train, scoring='neg_mean_squared_error'))
print(cross_val_score(gb_model, X_train, y_train, scoring='neg_mean_squared_error'))

/Users/adrian/anaconda3/envs/pydata/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[-18.86520326 -10.72103384  -7.02533944]


/Users/adrian/anaconda3/envs/pydata/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


[-19.09192861 -11.96982698  -6.93501085]
